### You can see at the bottom that I cross-validated my new and old models.  My OLD model was 4% better, which means I did a pretty good job initially.  It averaged an R squared of 84.7%, while my new model was 81.0% (across 6 folds).
### I want to talk about this though, because on the new 2014 data my old model didn't have any non-significance show up in any parameter, and that confuses me.
### I could keep engineering this and do transformations on the data, but the point is checking significance.  If I'm only dropping 3 or 4 percent by using only Population and Rape, the others are nearly insignificant.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
from scipy import stats
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 50)
pd.options.display.float_format = '{:.3f}'.format

%matplotlib inline

In [2]:
# Removing spaces in column names
features = pd.read_csv("feats.csv")
features.rename(columns={"Aggr Assault":"Assault",
                         "Property Crime":"Prop_Cr",
                         "Pop Squared":"Pop_Sqr"}, inplace=True)
features.head()

,Population,Viol_Cr,Murder,Rape,Robbery,Assault,Prop_Cr,Pop_Sqr
0,1861,0,0,0,0,0,12,3463321
1,2577,3,0,0,0,3,24,6640929
2,2846,3,0,0,0,3,16,8099716
3,97956,791,8,30,227,526,4090,9595377936
4,6388,23,0,3,4,16,223,40806544


In [3]:
# ORIGINAL
original = 'Prop_Cr ~ Population+Murder+Rape+Robbery+Assault+Pop_Sqr'
orig_model = smf.ols(formula=original, data=features).fit()

### Looks like Murder and Robbery are poor for predicting Prop_Cr.  I'm a little surprised that Pop_Sqr is significant though, but Population is, so it stands to reason that squaring it works.  Might be redundant.  I'll test dropping it and see how that changes my R-squared value.

### Interesting.  Robbery is insignificant in either case, so I'm dropping it.  Murder flip flops depending on my inclusion of Pop_Sqr or not, yet it's still 0.02 rather than much smaller, so I'm going to try dropping it.

In [4]:
orig_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Prop_Cr   R-squared:                       0.908
Model:                            OLS   Adj. R-squared:                  0.906
Method:                 Least Squares   F-statistic:                     558.7
Date:                Wed, 31 Oct 2018   Prob (F-statistic):          4.74e-173
Time:                        12:23:15   Log-Likelihood:                -2327.2
No. Observations:                 348   AIC:                             4668.
Df Residuals:                     341   BIC:                             4695.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -59.8017     14.978     -3.993      0.000     -89.262     -30.341
Population     0.0214      0.001     19.975      0.000       0.019       0.024
Murder        -9.4214     15.270     -0.617      0.538     -39.457      20.614
Rape          39.3314      4.355      9.032      0.000      30.766      47.897
Robbery       -0.2284      1.022     -0.223      0.823      -2.238       1.781
Assault        2.5851      0.633      4.086      0.000       1.341       3.829
Pop_Sqr    -9.473e-08   8.39e-09    -11.297      0.000   -1.11e-07   -7.82e-08
==============================================================================
Omnibus:                       56.188   Durbin-Watson:                   2.148
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              564.752
Skew:                           0.184   Prob(JB):                    2.32e-123
Kurtosis:                       9.230   Cond. No.                     4.91e+09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.91e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [5]:
# NEW MODEL
new = 'Prop_Cr ~ Population+Rape'
new_model = smf.ols(formula=new, data=features).fit()

### Murder and Robbery removed, model appears to still predict well.  I actually went back and included Violent Crime as a column, wondering if maybe it could act as PCA; it worsened the model.  Best I've found is: 
### Population + Rape

In [6]:
# Also looking at the (slight) difference between the summary types
new_model.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.870    
Dependent Variable: Prop_Cr          AIC:                4777.1622
Date:               2018-10-31 12:23 BIC:                4788.7188
No. Observations:   348              Log-Likelihood:     -2385.6  
Df Model:           2                F-statistic:        1163.    
Df Residuals:       345              Prob (F-statistic): 4.75e-154
R-squared:          0.871            Scale:              53158.   
-------------------------------------------------------------------
               Coef.   Std.Err.     t     P>|t|    [0.025    0.975]
-------------------------------------------------------------------
Intercept     14.4655   14.9159   0.9698  0.3328  -14.8721  43.8031
Population     0.0126    0.0007  17.5468  0.0000    0.0112   0.0140
Rape          61.4214    2.8266  21.7298  0.0000   55.8619  66.9810
------------------------------------------------------------------
Omnibus:             95.637       Durbin-Watson:          2.072   
Prob(Omnibus):       0.000        Jarque-Bera (JB):       3733.460
Skew:                0.203        Prob(JB):               0.000   
Kurtosis:            19.041       Condition No.:          33505   
==================================================================
* The condition number is large (3e+04). This might indicate
strong multicollinearity or other numerical problems.
"""

### Here's the original model cross-validating on the same 2013 data, just to see how it does.  Below that I'll try the new model.

In [7]:
from sklearn.model_selection import cross_val_score

# ORIGINAL MODEL, 2013 DATA
regr = linear_model.LinearRegression()
Y = features["Prop_Cr"]
X = features[["Population", "Murder", "Rape", "Robbery", "Assault", "Pop_Sqr"]]
regr.fit(X, Y)

scores = cross_val_score(regr, X, Y, cv=6)
print(f"Scores across {len(scores)} folds:\n{scores}\nOverall mean: {scores.mean()}")

Scores across 6 folds:
[0.92557069 0.71731736 0.76905888 0.85149616 0.86491103 0.93520581]
Overall mean: 0.843926653431895


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [8]:
# NEW MODEL, 2013 DATA
new_regr = linear_model.LinearRegression()
new_Y = features["Prop_Cr"]
new_X = features[["Population", "Rape"]]
new_regr.fit(new_X, new_Y)

new_scores = cross_val_score(new_regr, new_X, new_Y, cv=6)
print(f"Scores across {len(new_scores)} folds:\n{new_scores}\nOverall mean: {new_scores.mean()}")

Scores across 6 folds:
[0.91205699 0.63887995 0.78657279 0.79635956 0.88599312 0.83988352]
Overall mean: 0.809957656460211


### Now, let's try a new dataset, NY 2014.  I'm going to clean the data up as before and just import the cleaned file.

In [9]:
# Removing spaces in column names
feats2014 = pd.read_csv("feats2014.csv")
feats2014.rename(columns={"Aggr Assault":"Assault",
                         "Property Crime":"Prop_Cr",
                         "Pop Squared":"Pop_Sqr"}, inplace=True)
feats2014.head()

,Population,Murder,Rape,Robbery,Assault,Prop_Cr,Pop_Sqr
0,1851,0,0,0,0,11,3426201
1,2568,0,0,1,1,49,6594624
2,820,0,0,0,0,1,672400
3,2842,0,0,0,1,17,8076964
4,98595,8,54,237,503,3888,9720974025


### First test the original, then the revised model on the NEW 2014 DATA

In [10]:
# ORIGINAL MODEL, 2014 DATA
regr = linear_model.LinearRegression()
Y = feats2014["Prop_Cr"]
X = feats2014[["Population", "Murder", "Rape", "Robbery", "Assault", "Pop_Sqr"]]
regr.fit(X, Y)

scores = cross_val_score(regr, X, Y, cv=6)
print(f"Scores across {len(scores)} folds:\n{scores}\nOverall mean: {scores.mean()}")

Scores across 6 folds:
[0.94505837 0.73561793 0.8027137  0.806095   0.86783502 0.92747772]
Overall mean: 0.8474662908063245


In [11]:
# NEW MODEL, 2014 DATA
new_regr = linear_model.LinearRegression()
new_Y = feats2014["Prop_Cr"]
new_X = feats2014[["Population", "Rape"]]
new_regr.fit(new_X, new_Y)

new_scores = cross_val_score(new_regr, new_X, new_Y, cv=6)
print(f"Scores across {len(new_scores)} folds:\n{new_scores}\nOverall mean: {new_scores.mean()}")

Scores across 6 folds:
[0.91162853 0.73846358 0.80827315 0.72147823 0.87786821 0.80412732]
Overall mean: 0.8103065033351647


In [19]:
original = 'Prop_Cr ~ Population+Murder+Rape+Assault+Robbery+Pop_Sqr'
orig_model = smf.ols(formula=original, data=feats2014).fit()
orig_model.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.910    
Dependent Variable: Prop_Cr          AIC:                4921.2944
Date:               2018-10-31 12:26 BIC:                4948.6699
No. Observations:   369              Log-Likelihood:     -2453.6  
Df Model:           6                F-statistic:        619.4    
Df Residuals:       362              Prob (F-statistic): 5.81e-187
R-squared:          0.911            Scale:              35603.   
-------------------------------------------------------------------
             Coef.    Std.Err.     t     P>|t|    [0.025    0.975] 
-------------------------------------------------------------------
Intercept   -45.6668   13.7129  -3.3302  0.0010  -72.6338  -18.6998
Population    0.0187    0.0010  18.2762  0.0000    0.0167    0.0207
Murder      126.3392   13.9234   9.0739  0.0000   98.9583  153.7200
Rape         19.3445    1.9579   9.8802  0.0000   15.4942   23.1948
Assault       3.2291    0.5505   5.8661  0.0000    2.1466    4.3117
Robbery      -5.4968    1.0359  -5.3063  0.0000   -7.5340   -3.4597
Pop_Sqr      -0.0000    0.0000  -7.5252  0.0000   -0.0000   -0.0000
------------------------------------------------------------------
Omnibus:            107.798      Durbin-Watson:         2.050     
Prob(Omnibus):      0.000        Jarque-Bera (JB):      1298.689  
Skew:               0.846        Prob(JB):              0.000     
Kurtosis:           12.034       Condition No.:         4888215079
==================================================================
* The condition number is large (5e+09). This might indicate
strong multicollinearity or other numerical problems.
"""

In [13]:
new = 'Prop_Cr ~ Population+Rape'
new_model = smf.ols(formula=new, data=feats2014).fit()
new_model.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.844    
Dependent Variable: Prop_Cr          AIC:                5118.5307
Date:               2018-10-31 12:23 BIC:                5130.2631
No. Observations:   369              Log-Likelihood:     -2556.3  
Df Model:           2                F-statistic:        999.2    
Df Residuals:       366              Prob (F-statistic): 5.29e-149
R-squared:          0.845            Scale:              61413.   
-------------------------------------------------------------------
               Coef.   Std.Err.     t     P>|t|    [0.025    0.975]
-------------------------------------------------------------------
Intercept      8.6782   15.3279   0.5662  0.5716  -21.4636  38.8199
Population     0.0125    0.0008  16.2123  0.0000    0.0110   0.0140
Rape          34.8303    1.8604  18.7222  0.0000   31.1719  38.4886
------------------------------------------------------------------
Omnibus:              153.979      Durbin-Watson:         1.999   
Prob(Omnibus):        0.000        Jarque-Bera (JB):      4223.539
Skew:                 1.129        Prob(JB):              0.000   
Kurtosis:             19.420       Condition No.:         32627   
==================================================================
* The condition number is large (3e+04). This might indicate
strong multicollinearity or other numerical problems.
"""